In [1]:
# installing required libraries
import selenium 
from selenium import webdriver
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from itertools import repeat

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time

In [6]:
#defining attach function and empty lists that are required
air_company_list=[]
source_list=[]
destination_list=[]
departure_list=[]
arrival_list=[]
duration_list=[]
price_list=[]
date_and_week_list=[]
class_list=[]
stops_list=[]
def attach(x,y):
    for i in x:
        y.append(i.text)

In [7]:
places=['DEL']
place=['CCU','BOM','MAA']
date=['01','02','03','04','05','06','07','08','09',10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
cl=['E','PE','B']
# calling chrome driver
driver=webdriver.Chrome('chromedriver.exe')

In [8]:
for i in places:
    for j in place:
        for k in date:
            for l in cl:
                # getting the url
                url='https://www.makemytrip.com/flight/search?tripType=O&itinerary='+str(i)+'-'+str(j)+'-'+str(k)+'/05/2022&paxType=A-1_C-0_I-0&cabinClass='+str(l)+'&sTime=1651307443653&forwardFlowRequired=true&cmp=SEM|D|DF|G|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|529579609620&mpo=&semType='
                driver.get(url)
                try:
                    time.sleep(5)
                    driver.find_element_by_xpath('//button[@class="button buttonSecondry buttonBig fontSize12 relative"]').click()
                except NoSuchElementException:
                    pass
                time.sleep(5)
                SCROLL_PAUSE_TIME = 4
                driver.find_element_by_xpath('//span[@class="linkText ctaLink"]').click()
                time.sleep(2)
                date_and_week=driver.find_element_by_xpath('//p[@class="fontSize12 blackText boldFont appendBottom8"]').text
                # Get scroll height
                last_height = driver.execute_script("return document.body.scrollHeight")

                while True:
                    # Scroll down to bottom
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # Calculate new scroll height and compare with last scroll height
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                air_company=driver.find_elements_by_xpath('//span[@class="boldFont blackText airlineName"]')
                departure=driver.find_elements_by_xpath('//div[@class="flightTimeSection flexOne timeInfoLeft"]/div/p[1]')
                arrival=driver.find_elements_by_xpath('//div[@class="flightTimeSection flexOne timeInfoRight"]/div/p[1]')
                source=driver.find_elements_by_xpath('//div[@class="flightTimeSection flexOne timeInfoLeft"]/div/p[2]')
                destination=driver.find_elements_by_xpath('//div[@class="flightTimeSection flexOne timeInfoRight"]/div/p[2]')
                duration=driver.find_elements_by_xpath('//div[@class="stop-info flexOne"]/p')        
                stops=driver.find_elements_by_xpath('//p[@class="flightsLayoverInfo"]')
                price=driver.find_elements_by_xpath('//p[@class="blackText fontSize18 blackFont white-space-no-wrap"]')
                attach(air_company,air_company_list)
                attach(departure,departure_list)
                attach(arrival,arrival_list)
                attach(source,source_list)
                attach(destination,destination_list)
                attach(duration,duration_list)
                attach(stops,stops_list)
                class_list.extend(repeat(l,len(stops)))
                date_and_week_list.extend(repeat(date_and_week,len(stops)))
                attach(price,price_list)

In [10]:
#framing the data
flight_dataset=pd.DataFrame(list(zip(air_company_list,departure_list,arrival_list,source_list,destination_list,duration_list,stops_list,class_list,date_and_week_list,price_list)))
flight_dataset.columns=['Company','Departure time','Arrival time','Source','Destination','Duration time','Stops','Class','Date','Price']
flight_dataset.index+=1
flight_dataset

,Company,Departure time,Arrival time,Source,Destination,Duration time,Stops,Class,Date,Price
1,IndiGo,03:00,05:05,New Delhi,Kolkata,02 h 05 m,Non stop,E,"Sun, 1 May 22","₹ 7,319"
2,AirAsia,04:55,07:05,New Delhi,Kolkata,02 h 10 m,Non stop,E,"Sun, 1 May 22","₹ 7,319"
3,Go First,05:45,07:50,New Delhi,Kolkata,02 h 05 m,Non stop,E,"Sun, 1 May 22","₹ 7,319"
4,IndiGo,06:05,07:50,New Delhi,Kolkata,01 h 45 m,Non stop,E,"Sun, 1 May 22","₹ 7,319"
5,IndiGo,07:10,09:20,New Delhi,Kolkata,02 h 10 m,Non stop,E,"Sun, 1 May 22","₹ 7,319"
...,...,...,...,...,...,...,...,...,...,...
5819,Air India,09:45,12:40,New Delhi,Chennai,02 h 55 m,Non stop,B,"Mon, 30 May 22","₹ 27,864"
5820,Vistara,13:30,16:20,New Delhi,Chennai,02 h 50 m,Non stop,B,"Mon, 30 May 22","₹ 27,864"
5821,Vistara,16:15,19:30,New Delhi,Chennai,03 h 15 m,Non stop,B,"Mon, 30 May 22","₹ 27,864"
5822,Air India,17:25,20:15,New Delhi,Chennai,02 h 50 m,Non stop,B,"Mon, 30 May 22","₹ 27,864"


In [11]:
flight_dataset.iloc[3300]

Company                     Air India
Departure time                  21:15
Arrival time                    11:35
Source                      New Delhi
Destination                    Mumbai
Duration time               14 h 20 m
Stops             1 stop via Amritsar
Class                               B
Date                   Tue, 17 May 22
Price                        ₹ 35,532
Name: 3301, dtype: object

In [12]:
file_name = 'Flight_Ticket_Pricing_Data.xlsx'
flight_dataset.to_excel(file_name)

In [13]:
driver.close()